Remember that you need to get the email address from the JSON key and add that as a user in the Google Sheet
pulling-crm-data-to-master-dat@dealer-world-data.iam.gserviceaccount.com

In [205]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from datetime import datetime

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("dealer-world-data-16192d1e4c48.json", scope)
gc = gspread.authorize(creds)

client_list_original = gc.open_by_key('1_j31PwyFsTx0SPBIywvnOd7E3UbjUCFfAFIZkCOMr1g').sheet1.get_all_records()
ids = pd.DataFrame(client_list_original)
client_list = ids['Dealer Leads ID'].tolist()
print(client_list)



['1tnm4krKZK9KUuz_a0bCQ9I_TgCHQcewWIRh-9XY0JX4', '1pIFLDQjh3ArwsjGqZjbvC0eIRW2iDImv_2JiLyJG6ic', '1FY04jn-lE9rDSzUVHw41X-ZMm4Ao5bOQlFD-cnJVdKk', '1qJuX5Id_Aa5yBbbTXK7Qh9EejPZrYMVzzL-6hmm1HSI', '1ZjkqCzsJTgW5zBsBejf3PTkC_XR0dlscsTKkx7m9KB4', '1m40fVBOfzqqHJKK6sNhq8WGGxBdz3-5-apRUAfyWEdQ', '1EDl6blIaMbNkIm_vDh1fR4PFzxAbtmFx4wLjtTxrcjc', '1tmuvd-bWaEe-w6HSeEoS6-aJR19LbVXC6k5ioDKiXPk', '1OXCLC7_CId-g9gJGkWYNp2YGq1NdNOvSz3tl5Uu8zW4', '1vN5LKlordmU6PFMD8Ed8smT55CvN1VsvPRNZF6pVD64', '13Ag1u9ofQfF0-ZLDJRSFH2iFJLQJdDDF5S2Eu-PM1aE', '1lFUu-MzT4207SyTtRp3ncY7F0klWSFMtDRdnAjNvWC0', '1mj0R5TYIIZXa4_gCeq9mrgGQdskTP8HZl3z68zuXCCc', '1W2o2ctcZO9zfU63bnZ-praGcu2ZInhoxJ41hBw__cSY', '1QHRm7NJGoNzWvUYU9rl22JUsVBj-SyUPG7dohtftMc4', '1JLVWIC5bT_JOI5TsBO1Q8qx0c2A-TghmKbfTTMdmfjQ', '1IfU18GtoLFBHL6QWMdBR9XMc9VYM32bq-0IvrDmOf8A', '1cgt1U0zZHhZ_w4UIPmpleci3mxcdpTJMSKLc_i5nZpY', '1uhbpAi2UcTP-wTQCO8o5DhdKOhKJ0cnPAFCZ_ENei1w', '18pW__YqidJBu0dqT0U0oTHTWV0oLqXnqj2FokjrE3V0', '1I2BO14HHz5Mzkvt6L9CGMy6WBpV6iqpCqbLTm

In [206]:
master_df = pd.DataFrame()

for i in range(len(client_list)):
    try:
        sheet = gc.open_by_key(client_list[i]).worksheet("Dealer Leads")
        data = sheet.get_all_records()
        df = pd.DataFrame(data)
        master_df = master_df.append(df, ignore_index=True, sort=True)
    except:
        print(ids[ids['Dealer Leads ID'] == client_list[i]][['Client']]+' did not import correctly')

                              Client
25  Thrifty did not import correctly


In [207]:
#Dropping Empty Columns
master_df.drop(columns={''}, inplace=True)

In [208]:
master_df.dtypes

Avg. Response Time    object
Closing Percent       object
DealerName            object
Entry Date            object
Fresh Walk Ins        object
Internet Leads        object
Internet Set          object
Internet Show         object
Internet Sold         object
Phone Leads           object
Phone Set             object
Phone Show            object
Phone Sold            object
Showroom Visits       object
Units Sold            object
dtype: object

In [209]:
#Closing to Float
master_df['Closing Percent'] = master_df['Closing Percent'].replace('#DIV/0!','NaN').replace('','NaN').astype(float)

In [210]:
#Avg. Response Time to Float
master_df['Avg. Response Time'] = master_df['Avg. Response Time'].replace('','NaN').astype(float)

In [211]:
def str_to_int(column):
    master_df[column] = master_df[column].replace(' ','').replace('','0').astype(int)
    
int_columns = ['Fresh Walk Ins',
               'Internet Leads',
               'Internet Set',
               'Internet Show',
               'Internet Sold',
               'Phone Leads',
               'Phone Set',
               'Phone Show',
               'Phone Sold',
               'Showroom Visits',
               'Units Sold',
              ]

for i in int_columns:
    str_to_int(i)

In [212]:
master_df['Entry Date'] = pd.to_datetime(master_df['Entry Date'])

In [213]:
print("We have been tracking CRM info for "+str((master_df['Entry Date'].max() - master_df['Entry Date'].min()).days)+" days.")

We have been tracking CRM info for 378 days.


In [214]:
master_df.resample('M').mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'